# Logsignature example
This notebook is based on the examples from the `torchcde` package by Kidger and Morrill which can be found at
[https://github.com/patrick-kidger/torchcde](https://github.com/patrick-kidger/torchcde).
Further information about the techniques described in this notebook can be found

> Morrill, J., Salvi, C., Kidger, P., Foster, J. and Lyons, T., 2020.
  Neural rough differential equations for long time series.
  arXiv preprint arXiv:[2009.08295](https://arxiv.org/abs/2009.08295)

> Morrill, J., Kidger, P., Yang, L. and Lyons, T., 2021.
  Neural Controlled Differential Equations for Online Prediction Tasks.
  arXiv preprint arXiv:[2106.11028](https://arxiv.org/abs/2106.11028).

> Kidger, P., Foster, J., Li, X., Oberhauser, H. and Lyons, T., 2021.
  Neural sdes as infinite-dimensional gans.
  arXiv preprint arXiv:[2102.03657](https://arxiv.org/abs/2102.03657).

In this notebook we code up a Neural CDE using the log-ode method for a long time series thus becoming a Neural RDE.
We will only describe the differences from that example.


## Set up the notebook


### Install dependencies
This notebook uses PyTorch along with

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/patrick-kidger/torchcde.git (to revision 4620b4b8bfb08416f4871c9e76510b69439ee916) to /tmp/pip-install-sjadnydl/torchcde_37b73a691fa5457fb8fbb7e106454be4
  Running command git clone -q https://github.com/patrick-kidger/torchcde.git /tmp/pip-install-sjadnydl/torchcde_37b73a691fa5457fb8fbb7e106454be4
  Running command git rev-parse -q --verify 'sha^4620b4b8bfb08416f4871c9e76510b69439ee916'
  Running command git fetch -q https://github.com/patrick-kidger/torchcde.git 4620b4b8bfb08416f4871c9e76510b69439ee916
  Cloning https://github.com/google-research/torchsde.git to /tmp/pip-install-sjadnydl/torchsde_44060aa4901d47d4abe30cb2dad4b661
  Running command git clone -q https://github.com/google-research/torchsde.git /tmp/pip-install-sjadnydl/torchsde_44060aa4901d47d4abe30cb2dad4b661
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - done


In [2]:
import math
import time
import torch
import torchcde

We use the `CDEFunc` and `NeuralCDE` classes, and the `get_data` function defined in the _time series classificiation_
notebook.

In [3]:
class CDEFunc(torch.nn.Module):
    def __init__(self, input_channels, hidden_channels):
        ######################
        # input_channels is the number of input channels in the data X. (Determined by the data.)
        # hidden_channels is the number of channels for z_t. (Determined by you!)
        ######################
        super(CDEFunc, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels

        self.linear1 = torch.nn.Linear(hidden_channels, 128)
        self.linear2 = torch.nn.Linear(128, input_channels * hidden_channels)

    ######################
    # For most purposes the t argument can probably be ignored; unless you want your CDE to behave differently at
    # different times, which would be unusual. But it's there if you need it!
    ######################
    def forward(self, t, z):
        # z has shape (batch, hidden_channels)
        z = self.linear1(z)
        z = z.relu()
        z = self.linear2(z)
        ######################
        # Easy-to-forget gotcha: Best results tend to be obtained by adding a final tanh nonlinearity.
        ######################
        z = z.tanh()
        ######################
        # Ignoring the batch dimension, the shape of the output tensor must be a matrix,
        # because we need it to represent a linear map from R^input_channels to R^hidden_channels.
        ######################
        z = z.view(z.size(0), self.hidden_channels, self.input_channels)
        return z


class NeuralCDE(torch.nn.Module):
    def __init__(self, input_channels, hidden_channels, output_channels, interpolation="cubic"):
        super(NeuralCDE, self).__init__()

        self.func = CDEFunc(input_channels, hidden_channels)
        self.initial = torch.nn.Linear(input_channels, hidden_channels)
        self.readout = torch.nn.Linear(hidden_channels, output_channels)
        self.interpolation = interpolation

    def forward(self, coeffs):
        if self.interpolation == 'cubic':
            X = torchcde.NaturalCubicSpline(coeffs)
        elif self.interpolation == 'linear':
            X = torchcde.LinearInterpolation(coeffs)
        else:
            raise ValueError("Only 'linear' and 'cubic' interpolation methods are implemented.")

        ######################
        # Easy to forget gotcha: Initial hidden state should be a function of the first observation.
        ######################
        X0 = X.evaluate(X.interval[0])
        z0 = self.initial(X0)

        ######################
        # Actually solve the CDE.
        ######################
        z_T = torchcde.cdeint(X=X,
                              z0=z0,
                              func=self.func,
                              t=X.interval)

        ######################
        # Both the initial value and the terminal value are returned from cdeint; extract just the terminal value,
        # and then apply a linear map.
        ######################
        z_T = z_T[:, 1]
        pred_y = self.readout(z_T)
        return pred_y


def get_data(num_timepoints=100):
    t = torch.linspace(0., 4 * math.pi, num_timepoints)

    start = torch.rand(128) * 2 * math.pi
    x_pos = torch.cos(start.unsqueeze(1) + t.unsqueeze(0)) / (1 + 0.5 * t)
    x_pos[:64] *= -1
    y_pos = torch.sin(start.unsqueeze(1) + t.unsqueeze(0)) / (1 + 0.5 * t)
    x_pos += 0.01 * torch.randn_like(x_pos)
    y_pos += 0.01 * torch.randn_like(y_pos)
    ######################
    # Easy to forget gotcha: time should be included as a channel; Neural CDEs need to be explicitly told the
    # rate at which time passes. Here, we have a regularly sampled dataset, so appending time is pretty simple.
    ######################
    X = torch.stack([t.unsqueeze(0).repeat(128, 1), x_pos, y_pos], dim=2)
    y = torch.zeros(128)
    y[:64] = 1

    perm = torch.randperm(128)
    X = X[perm]
    y = y[perm]

    ######################
    # X is a tensor of observations, of shape (batch=128, sequence=100, channels=3)
    # y is a tensor of labels, of shape (batch=128,), either 0 or 1 corresponding to anticlockwise or clockwise respectively.
    ######################
    return X, y

Now we can define a function that will train the model and evaluate the performance on our data set using logsignatures
up to a specified depth.

In [4]:
def train_and_evaluate(train_X, train_y, test_X, test_y, depth, num_epochs, window_length):
    # Time the training process
    start_time = time.time()

    # Logsignature computation step
    train_logsig = torchcde.logsig_windows(train_X, depth, window_length=window_length)
    print("Logsignature shape: {}".format(train_logsig.size()))

    model = NeuralCDE(
        input_channels=train_logsig.size(-1), hidden_channels=8, output_channels=1, interpolation="linear"
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

    train_coeffs = torchcde.linear_interpolation_coeffs(train_logsig)

    train_dataset = torch.utils.data.TensorDataset(train_coeffs, train_y)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch_coeffs, batch_y = batch
            pred_y = model(batch_coeffs).squeeze(-1)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(pred_y, batch_y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Epoch: {}   Training loss: {}".format(epoch, loss.item()))

    # Remember to compute the logsignatures of the test data too!
    test_logsig = torchcde.logsig_windows(test_X, depth, window_length=window_length)
    test_coeffs = torchcde.linear_interpolation_coeffs(test_logsig)
    pred_y = model(test_coeffs).squeeze(-1)
    binary_prediction = (torch.sigmoid(pred_y) > 0.5).to(test_y.dtype)
    prediction_matches = (binary_prediction == test_y).to(test_y.dtype)
    proportion_correct = prediction_matches.sum() / test_y.size(0)
    print("Test Accuracy: {}".format(proportion_correct))

    # Total time
    elapsed = time.time() - start_time

    return proportion_correct, elapsed


In [5]:
NUM_EPOCHS = 10

Here we load a high frequency version of the spiral data using in `torchcde.example`. Each sample contains 5000
time points. This is too long to sensibly expect a Neural CDE to handle, training time will be very long and it
will struggle to remember information from early on in the sequence.

In [6]:
num_timepoints = 5000
train_X, train_y = get_data(num_timepoints=num_timepoints)
test_X, test_y = get_data(num_timepoints=num_timepoints)


We test the model over logsignature depths 1, 2, and 3, with a window length of 50. This reduces the effective
length of the path to just 100. The only change is an application of `torchcde.logsig_windows`

The raw signal has 3 input channels. Taking logsignatures of depths 1, 2, and 3 results in a path of logsignatures
of dimension 3, 6, and 14 respectively. We see that higher logsignature depths contain more information about the
path over the intervals, at a cost of increased numbers of channels.

In [7]:
depths = [1, 2, 3]
window_length = 50
accuracies = []
training_times = []
for depth in depths:
    print(f'Running for logsignature depth: {depth}')
    acc, elapsed = train_and_evaluate(
        train_X, train_y, test_X, test_y, depth, NUM_EPOCHS, window_length
    )
    training_times.append(elapsed)
    accuracies.append(acc)


Running for logsignature depth: 1
Logsignature shape: torch.Size([128, 101, 3])
Epoch: 0   Training loss: 2.5779426097869873
Epoch: 1   Training loss: 0.9518626928329468
Epoch: 2   Training loss: 0.6158618927001953
Epoch: 3   Training loss: 0.678015410900116
Epoch: 4   Training loss: 0.6150668263435364
Epoch: 5   Training loss: 0.521719753742218
Epoch: 6   Training loss: 0.36672350764274597
Epoch: 7   Training loss: 0.47279104590415955
Epoch: 8   Training loss: 0.4549952447414398
Epoch: 9   Training loss: 0.28105735778808594
Test Accuracy: 0.953125
Running for logsignature depth: 2
Logsignature shape: torch.Size([128, 101, 6])
Epoch: 0   Training loss: 0.4106109142303467
Epoch: 1   Training loss: 1.03876793384552
Epoch: 2   Training loss: 0.7270088791847229
Epoch: 3   Training loss: 0.8129279613494873
Epoch: 4   Training loss: 0.63377445936203
Epoch: 5   Training loss: 0.554548978805542
Epoch: 6   Training loss: 0.3670040965080261
Epoch: 7   Training loss: 0.24859601259231567
Epoch: 8 

Finally, log the results to the console for a comparison

In [8]:
print("Final results")
for acc, elapsed, depth in zip(accuracies, training_times, depths):
    print(
        f"Depth: {depth}\n\tAccuracy on test set: {acc*100:.1f}%\n\tTime per epoch: {elapsed/NUM_EPOCHS:.1f}s"
    )


Final results
Depth: 1
	Accuracy on test set: 95.3%
	Time per epoch: 3.8s
Depth: 2
	Accuracy on test set: 100.0%
	Time per epoch: 3.5s
Depth: 3
	Accuracy on test set: 100.0%
	Time per epoch: 5.7s
